<a href="https://colab.research.google.com/github/gyanasluitel/Music_Genre_Classification/blob/main/preparing_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [41]:
import os
import pandas as pd
import numpy as npm

In [ ]:
import librosa

In [ ]:
import math
import glob

In [ ]:
!pip install -q kaggle

In [ ]:
#Upload kaggle.json file before running this cell
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d andradaolteanu/gtzan-dataset-music-genre-classification

 99% 1.21G/1.21G [00:21<00:00, 63.1MB/s]
100% 1.21G/1.21G [00:21<00:00, 61.3MB/s]


In [ ]:
!unzip gtzan-dataset-music-genre-classification.zip

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
path = '/content/drive/My Drive/Sub_Data'
path

'/content/drive/My Drive/Sub_Data'

In [ ]:
os.listdir(path)

['images_original', 'genres_original']

In [ ]:
path = os.getcwd()
DATASET_PATH = os.path.join(path, 'Data/genres_original')
DATASET_PATH

'/content/Data/genres_original'

In [57]:
def prepare_dataset(datapath,n_mfcc=13, n_fft=248,hop_length=512, num_segments=5):
    """
    extracting mfcc features from audio 
    """
    SAMPLE_RATE = 22050
    DURATION = 30
    SAMPLES_PER_TRACK = SAMPLE_RATE * DURATION

    #preparing header for dataframe columns
    header = ''
    for i in range(1, 14):
        header += f' mfcc{i}'
    header = header.split()
    header
    
    final_dataframe = pd.DataFrame()
    num_samples_per_segment =int(SAMPLES_PER_TRACK  / num_segments)
    expected_num_mfcc_vectors_per_segment = math.ceil(num_samples_per_segment / hop_length)
    
    for dirpath, dirnames, filenames in os.walk(DATASET_PATH):
          dirnames.sort()
          if dirpath is not DATASET_PATH:
            dirpath_components = dirpath.split('/')
            semantic_label = dirpath_components[-1]
            # print(f"Dirpath Components: {dirpath_components}")
            # print(f'Semantic Label: {semantic_label}')
            print(f"\nProcessing: {semantic_label}")

            single_file_dataframe = pd.DataFrame()
            #processing files of a specific genre
            for file in filenames:
                  file_path = os.path.join(dirpath,file)
                  # print(f"File Path: {file_path}")
                  label = file_path.split('/')[-2]
                  # print(f"Label: {label}")

                  try:
                    signal, sr = librosa.load(file_path, sr = SAMPLE_RATE)

                  #process segments extracting mfcc and storing data
                    for s in range(num_segments):
                      start_sample = num_samples_per_segment * s
                      finish_sample = start_sample + num_samples_per_segment

                      mfcc = librosa.feature.mfcc(signal[start_sample:finish_sample],
                                                  sr = sr,
                                                  n_mfcc = n_mfcc,
                                                  n_fft = n_fft,
                                                  hop_length = hop_length)
                      mfcc = mfcc.T

                      #store mfcc for segment if it has the expected length
                      if len(mfcc) == expected_num_mfcc_vectors_per_segment:
                          segment_dataframe = pd.DataFrame(data = mfcc,
                                      columns =header, 
                                      index = range(len(mfcc)))
                          segment_dataframe['Labels'] = [label] * mfcc.shape[0]
                          
                          #concatenating dataframe of segement into the single file dataframe
                          single_file_dataframe = pd.concat([single_file_dataframe, segment_dataframe],
                                                      axis=0,
                                                      sort=False,
                                                      ignore_index = True)
                  except Exception as e:
                    print(f"Error Message: {e}")
                  # break
            final_dataframe  = pd.concat([final_dataframe, single_file_dataframe],
                              axis=0,
                              sort=False,
                              ignore_index = True)  
    return final_dataframe




In [58]:
df = prepare_dataset(DATASET_PATH)


Processing: blues


/usr/local/lib/python3.7/dist-packages/librosa/filters.py:239: UserWarning: Empty filters detected in mel frequency basis. Some channels will produce empty responses. Try increasing your sampling rate (and fmax) or reducing n_mels.
  "Empty filters detected in mel frequency basis. "



Processing: classical

Processing: country

Processing: disco

Processing: hiphop

Processing: jazz


/usr/local/lib/python3.7/dist-packages/librosa/core/audio.py:162: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn("PySoundFile failed. Trying audioread instead.")


Error Message: 


/usr/local/lib/python3.7/dist-packages/librosa/filters.py:239: UserWarning: Empty filters detected in mel frequency basis. Some channels will produce empty responses. Try increasing your sampling rate (and fmax) or reducing n_mels.
  "Empty filters detected in mel frequency basis. "



Processing: metal

Processing: pop

Processing: reggae

Processing: rock


In [59]:
df['Labels'].unique()

array(['blues', 'classical', 'country', 'disco', 'hiphop', 'jazz',
       'metal', 'pop', 'reggae', 'rock'], dtype=object)

In [60]:
print(f"Df shape: {df.shape}")
df.head()
# df

Df shape: (1293187, 14)


,mfcc1,mfcc2,mfcc3,mfcc4,mfcc5,mfcc6,mfcc7,mfcc8,mfcc9,mfcc10,mfcc11,mfcc12,mfcc13,Labels
0,-433.884308,15.913300,-25.085659,20.029894,34.291142,-12.760870,-19.364769,-24.480751,-4.493195,-4.777666,-10.788865,-3.239865,-11.257139,blues
1,-547.664917,94.959518,-70.481400,14.092028,35.109077,11.970919,-12.464500,0.716512,-34.472649,-3.625931,15.496975,-17.343588,-15.245530,blues
2,-548.248291,87.378922,-38.207825,13.292370,24.732548,23.172117,-6.250585,-4.227919,-48.555984,9.987759,6.117764,-18.813776,-29.227020,blues
3,-535.345093,97.110458,-65.098969,1.792318,31.079037,23.876558,-12.207192,-17.244558,-41.387501,19.903538,16.261974,-24.469753,-25.385651,blues
4,-506.649414,81.980164,-98.566498,-2.019200,49.388485,18.573458,-37.221394,-5.270992,-27.270142,21.190189,15.351100,-10.902548,-24.929514,blues
